# opencv video example

In [1]:
import cv2
from PIL import Image


In [2]:
cap = cv2.VideoCapture('armada_vs_hbox.mp4')

In [3]:
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(n_frames)

19722


In [4]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 99)
res, frame = cap.read()
Image.fromarray(frame[...,::-1], 'RGB').resize((256, 256)).show()

In [5]:
def resize(im, desired_size, maintain_aspect_ratio=True, padding_color=(0, 0, 0)):
    # adapted from https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/#using-opencv
    old_size = im.shape[:2] # old_size is in (height, width) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format

    im = cv2.resize(im, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=padding_color)

    return new_im

In [6]:
print(frame.shape)
print(resize(frame, 256).shape)

(720, 1280, 3)
(256, 256, 3)


In [33]:
import random
import numpy as np
from pathlib import Path

def sample_frames_from_video(video, k, technique='random'):
    if isinstance(video, str):
        cap = cv2.VideoCapture(video)
    elif isinstance(video, cv2.VideoCapture):
        cap = video
    else:
        raise TypeError(f'Illegal type f{video.__class__.__name__} for argument `video`, must be str or cv2.VideoCapture.')
    
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    k = min(k, n_frames)
    
    if technique == 'first':
        idxs = list(range(k))
    elif technique == 'random':
        idxs = random.sample(range(n_frames), k)
    
    frames = []
    for idx in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        _, frame = cap.read()
        frames.append(frame)
        
    return idxs, np.array(frames)

In [34]:
idxs, frames = sample_frames_from_video('armada_vs_hbox.mp4', 10)
print(idxs)
print(frames.shape)

[12931, 8957, 9308, 13901, 9584, 11319, 15670, 18072, 10164, 5897]
(10, 720, 1280, 3)


In [35]:
import random

def random_bbox(im):
    shape = im.shape
    x_min, x_max = sorted([random.randrange(0, im.shape[0]+1) for _ in range(2)])
    y_min, y_max = sorted([random.randrange(0, im.shape[1]+1) for _ in range(2)])
    return (x_min, x_max), (y_min, y_max)

def get_bboxes(im, window):
    shape = w, h = im.shape[:2]
    bboxes = []
    for x0 in range(0, w, window):
        for y0 in range(0, h, window):
            bboxes.append(((x0, x0+window), (y0, y0+window)))
    return bboxes

def crop(im, x_min, x_max, y_min, y_max):
    return im[x_min:x_max, y_min:y_max]


In [49]:
windows = [512, 384, 256, 192, 128, 96, 64, 48, 32, 24, 16]
bboxes = []
for window in windows:
    bboxes.extend(get_bboxes(im, window))
print(len(bboxes))

2019


In [47]:
im = frames[0]
(x_min, x_max), (y_min, y_max) = bboxes[0]
im = crop(im, x_min, x_max, y_min, y_max)

In [54]:
Image.fromarray(im, 'RGB').show()

In [9]:
width, height, channels = frames.shape[1:]

from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(frames, test_size=0.1, random_state=42)

In [10]:
X_train.shape

(115, 256, 256, 3)

In [11]:
X_test.shape

(13, 256, 256, 3)

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(featurewise_center=True,
                             featurewise_std_normalization=True,
                             zoom_range=0.1)
datagen.fit(X_train)
train_iterator = datagen.flow(X_train, X_train, batch_size=32)
test_iterator = datagen.flow(X_test, X_test, batch_size=64)
print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))

Batches train=4, test=1


https://blog.keras.io/building-autoencoders-in-keras.html

https://keras.io/examples/mnist_denoising_autoencoder/